In [1]:
import pandas as pd
import panel as pn
import geopandas as gpd
from trino.dbapi import connect
# import folium
import param
import os



In [2]:
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [3]:
# Trino connection configuration
TRINO_HOST = os.environ.get("TRINO_HOST", "localhost")
TRINO_PORT = os.environ.get("TRINO_PORT", 8080)
TRINO_USER = os.environ.get("TRINO_USER", "teehr")
TRINO_CATALOG = os.environ.get("TRINO_CATALOG", "iceberg")
TRINO_SCHEMA = os.environ.get("TRINO_SCHEMA", "teehr")

In [4]:
pn.extension('tabulator', design="material", sizing_mode="stretch_width")

In [5]:
class DashboardState(param.Parameterized):
    locations_gdf = param.ClassSelector(class_=gpd.GeoDataFrame, default=None)

state = DashboardState()

In [6]:

def get_trino_connection():
    """Establishes and returns a Trino database connection."""
    conn = connect(
        host=TRINO_HOST,
        user=TRINO_USER,
        catalog=TRINO_CATALOG,
        schema=TRINO_SCHEMA,
        http_scheme='http',
        port=TRINO_PORT,
        # For production, add authentication:
        # auth=BasicAuthentication("username", "password")
    )
    return conn


In [8]:
def get_unique_location_prefixes():
    """"""
    conn = get_trino_connection()
    query = """
    WITH prefixes AS (
        SELECT 
            split(id, '-')[1] AS id_prefix,
            split(id, '-')[2] AS id_suffix
        FROM iceberg.teehr.locations
    )
    SELECT id_prefix as primary_location_id_prefix,
        count(*) AS location_count
    FROM prefixes
    GROUP BY id_prefix
    """
    df = pd.read_sql(query, conn)
    return df

get_unique_location_prefixes()

,primary_location_id_prefix,location_count
0,usgs,13654


In [9]:
def get_unique_crosswalks_by_prefix():
    """Gets unique crosswalks grouped by primary and secondary location id prefixes."""
    conn = get_trino_connection()
    query = """
    WITH prefixes AS (
        SELECT 
            split(primary_location_id, '-')[1] AS primary_id_prefix,
            split(primary_location_id, '-')[2] AS primary_id_suffix,
            split(secondary_location_id, '-')[1] AS secondary_id_prefix,
            split(secondary_location_id, '-')[2] AS secondary_id_suffix,
            lc.*
        FROM iceberg.teehr.location_crosswalks lc
    )
    SELECT 
        primary_id_prefix, secondary_id_prefix,
        count(*) AS location_count
    FROM prefixes
    GROUP BY primary_id_prefix, secondary_id_prefix
    """
    df = pd.read_sql(query, conn)
    # gdf = df_to_gdf(df)
    return df

get_unique_crosswalks_by_prefix()

,primary_id_prefix,secondary_id_prefix,location_count
0,usgs,nwm30,8313


In [10]:
def get_unique_primary_timeseries_configurations():
    """Gets unique primary timeseries configurations grouped by location id prefixes."""
    conn = get_trino_connection()
    query = """
    WITH prefixes AS (
        SELECT 
            split(location_id, '-')[1] AS primary_id_prefix,
            split(location_id, '-')[2] AS primary_id_suffix,
            pt.*
        FROM iceberg.teehr.primary_timeseries pt
    )
    SELECT 
        primary_id_prefix, configuration_name, variable_name, unit_name,
        count(*) AS timeseries_value_count
    FROM prefixes
    GROUP BY primary_id_prefix, configuration_name, variable_name, unit_name
    """
    df = pd.read_sql(query, conn)
    # gdf = df_to_gdf(df)
    return df

get_unique_primary_timeseries_configurations()

,primary_id_prefix,configuration_name,variable_name,unit_name,timeseries_value_count
0,usgs,usgs_observations,streamflow_hourly_inst,m^3/s,1560240516


In [10]:
def get_unique_secondary_timeseries_configurations():
    """Gets unique secondary timeseries configurations grouped by location id prefixes."""
    conn = get_trino_connection()
    query = """
    WITH prefixes AS (
        SELECT 
            split(location_id, '-')[1] AS secondary_id_prefix,
            split(location_id, '-')[2] AS secondary_id_suffix,
            st.*
        FROM iceberg.teehr.secondary_timeseries st
    )
    SELECT 
        secondary_id_prefix, configuration_name, variable_name, unit_name,
        count(*) AS timeseries_value_count
    FROM prefixes
    GROUP BY secondary_id_prefix, configuration_name, variable_name, unit_name
    """
    df = pd.read_sql(query, conn)
    # gdf = df_to_gdf(df)
    return df

get_unique_secondary_timeseries_configurations()

,secondary_id_prefix,configuration_name,variable_name,unit_name,timeseries_value_count
0,nwm30,nwm30_retrospective,streamflow_hourly_inst,m^3/s,3061335120


In [14]:
def get_primary_timeseries_statistics_by_location():
    """Gets primary timeseries statistics for a specific location."""
    conn = get_trino_connection()
    query = """
    SELECT 
        location_id, configuration_name, variable_name, unit_name,
        count(*) AS timeseries_value_count,
        min(value_time) as start_time,
        max(value_time) as end_time,
        date_diff('hour', min(value_time), max(value_time)) as hours_between,
        date_diff('day', min(value_time), max(value_time)) as days_between
    FROM iceberg.teehr.primary_timeseries
    GROUP BY location_id, configuration_name, variable_name, unit_name
    """
    df = pd.read_sql(query, conn)
    # gdf = df_to_gdf(df)
    return df

get_primary_timeseries_statistics_by_location()

,location_id,configuration_name,variable_name,unit_name,timeseries_value_count,start_time,end_time,hours_between,days_between
0,usgs-01011000,usgs_observations,streamflow_hourly_inst,m^3/s,182270,1987-12-02 05:00:00+00:00,2023-12-31 19:00:00+00:00,316286,13178
1,usgs-01014000,usgs_observations,streamflow_hourly_inst,m^3/s,173227,1988-10-05 04:00:00+00:00,2023-12-31 23:00:00+00:00,308899,12870
2,usgs-01015800,usgs_observations,streamflow_hourly_inst,m^3/s,170227,1990-10-01 05:00:00+00:00,2023-12-31 23:00:00+00:00,291474,12144
3,usgs-01017290,usgs_observations,streamflow_hourly_inst,m^3/s,88430,2008-06-01 04:00:00+00:00,2023-12-31 21:00:00+00:00,136601,5691
4,usgs-04024430,usgs_observations,streamflow_hourly_inst,m^3/s,196064,1986-10-01 06:00:00+00:00,2023-11-22 04:00:00+00:00,325582,13565
...,...,...,...,...,...,...,...,...,...
8136,usgs-07097000,usgs_observations,streamflow_hourly_inst,m^3/s,5710,2005-10-01 06:00:00+00:00,2006-09-30 05:00:00+00:00,8735,363
8137,usgs-04293700,usgs_observations,streamflow_hourly_inst,m^3/s,15697,2009-08-09 04:00:00+00:00,2011-10-19 03:00:00+00:00,19223,800
8138,usgs-385202111121601,usgs_observations,streamflow_hourly_inst,m^3/s,11005,2015-04-29 06:00:00+00:00,2016-10-19 05:00:00+00:00,12935,538
8139,usgs-03501500,usgs_observations,streamflow_hourly_inst,m^3/s,18468,2013-12-21 05:00:00+00:00,2016-02-01 04:00:00+00:00,18527,771


In [15]:
def get_secondary_timeseries_statistics_by_location():
    """Gets secondary timeseries statistics for a specific location."""
    conn = get_trino_connection()
    query = """
    WITH prefixes AS (
        SELECT 
            lc.primary_location_id,
            st.*
        FROM iceberg.teehr.secondary_timeseries st
        JOIN iceberg.teehr.location_crosswalks lc
            ON st.location_id = lc.secondary_location_id
    )
    SELECT 
        primary_location_id, location_id as secondary_location_id, configuration_name, variable_name, unit_name,
        count(*) AS timeseries_value_count,
        min(value_time) as start_time,
        max(value_time) as end_time,
        date_diff('hour', min(value_time), max(value_time)) as hours_between,
        date_diff('day', min(value_time), max(value_time)) as days_between
    FROM prefixes
    GROUP BY primary_location_id, location_id, configuration_name, variable_name, unit_name
    """
    df = pd.read_sql(query, conn)
    # gdf = df_to_gdf(df)
    return df

get_secondary_timeseries_statistics_by_location()

DatabaseError: Execution failed on sql: 
    WITH prefixes AS (
        SELECT 
            lc.primary_location_id,
            st.*
        FROM iceberg.teehr.secondary_timeseries st
        JOIN iceberg.teehr.location_crosswalks lc
            ON st.location_id = lc.secondary_location_id
    )
    SELECT 
        primary_location_id, location_id as secondary_location_id, configuration_name, variable_name, unit_name,
        count(*) AS timeseries_value_count,
        min(value_time) as start_time,
        max(value_time) as end_time,
        date_diff('hour', min(value_time), max(value_time)) as hours_between,
        date_diff('day', min(value_time), max(value_time)) as days_between
    FROM prefixes
    GROUP BY primary_location_id, location_id, configuration_name, variable_name, unit_name
    
Input is a zero-length, empty document: line 1 column 1 (char 0)
unable to rollback

In [ ]:
unique_location_prefixes_tabulator = pn.widgets.Tabulator(
    get_unique_location_prefixes(), 
    theme="bootstrap", 
    pagination="remote", 
    page_size=10
)
unique_primary_timeseries_configurations_tabulator = pn.widgets.Tabulator(
    get_unique_primary_timeseries_configurations(), 
    theme="bootstrap", 
    pagination="remote", 
    page_size=10
)
unique_secondary_timeseries_configurations_tabulator = pn.widgets.Tabulator(
    get_unique_secondary_timeseries_configurations(), 
    theme="bootstrap", 
    pagination="remote", 
    page_size=10
)
unique_crosswalks_by_prefix_tabulator = pn.widgets.Tabulator(
    get_unique_crosswalks_by_prefix(), 
    theme="bootstrap",
    pagination="remote",
    page_size=10
)
primary_timeseries_statistics_by_location_tabulator = pn.widgets.Tabulator(
    get_primary_timeseries_statistics_by_location(), 
    theme="bootstrap",
    pagination="remote",
    page_size=10
)
secondary_timeseries_statistics_by_location_tabulator = pn.widgets.Tabulator(
    get_secondary_timeseries_statistics_by_location(), 
    theme="bootstrap",
    pagination="remote",
    page_size=10
)

# locations_widget = pn.widgets.Select(
#     name="location",
#     options=list(state.metrics_gdf['primary_location_id']),
#     value=state.location_id,
# )

sidebar = pn.Column(
    pn.pane.Markdown(f"""
        ### 🔧 Database Connection Information
        ```
        Host: {TRINO_HOST}
        Port: {TRINO_PORT}
        User: {TRINO_USER}
        Catalog: {TRINO_CATALOG}
        Schema: {TRINO_SCHEMA}
        ```
    """),
    "---",
)

main = pn.Column(
    pn.pane.Markdown("""
        # Data Summary Dashboard
        ### This dashboard is a placeholder for various data summaries and statistics related to the TEEHR warehouse.  
        ### We will update this as we identify useful summaries to include.
    """),
    pn.pane.Markdown("""
        ## 📊 Unique Location Prefixes
        ### This table provides a summary of unique location ID prefixes stored in the TEEHR warehouse.
    """),
    unique_location_prefixes_tabulator,
    "---",
    pn.pane.Markdown("""
        ## 🔄 Crosswalks by Location Prefixes
        ### This table provides a summary of unique crosswalks grouped by primary and secondary location ID prefixes.
    """),
    unique_crosswalks_by_prefix_tabulator,
    "---",
    pn.pane.Markdown("""
        ## 📈 Primary Time Series Configurations
        ### This table provides a summary of unique primary time series configurations stored in the TEEHR warehouse.
    """),
    unique_primary_timeseries_configurations_tabulator,
    "---",
    pn.pane.Markdown("""
        ## 📈 Secondary Time Series Configurations
        ### This table provides a summary of unique secondary time series configurations stored in the TEEHR warehouse.
    """),
    unique_secondary_timeseries_configurations_tabulator,
    "---",
    pn.pane.Markdown("""
        ## 📊 Primary Time Series Statistics by Location
        ### This table provides a summary of primary time series statistics grouped by location.
    """),
    primary_timeseries_statistics_by_location_tabulator,
    "---",
    pn.pane.Markdown("""
        ## 📊 Secondary Time Series Statistics by Location
        ### This table provides a summary of secondary time series statistics grouped by location.
    """),
    secondary_timeseries_statistics_by_location_tabulator,
    "---"
)


pn.template.MaterialTemplate(
    site="TEEHR",
    title="Data Summary Dashboard",
    sidebar=[sidebar],
    main=[main],
).servable()

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-13548739216] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', theme='bootstrap', value=  id_prefix  c...)
        [1] Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', theme='bootstrap', value=  primary_id_prefix c...)
        [2] Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', theme='bootstrap', value=  secondary_id_prefix  ...)
        [3] Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', theme='bootstrap', value=  primary_id_prefix s...)
    [nav-13547842464] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [1] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')